In [1]:
import sys
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
import src.utils as utils
import src.myenv as myenv
import pandas as pd
import glob
import os
import zipfile

symbols = 'BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,ADAUSDT,DOGEUSDT,SOLUSDT,TRXUSDT,DOTUSDT,MATICUSDT,LTCUSDT,BCHUSDT,AVAXUSDT,XLMUSDT,LINKUSDT,XMRUSDT,UNIUSDT,ATOMUSDT'.split(',')
intervals = '1m,5m,15m,30m,1h'.split(',')

for symbol in symbols:
	for interval in intervals:
		print(f'Process {symbol}_{interval}')
		file_zip_list = glob.glob(os.path.join(f'{myenv.datadir}2/{symbol}/{interval}/', '*.zip'))
		for file_zip in file_zip_list:
			with zipfile.ZipFile(file_zip, 'r') as zip_ref:
				zip_ref.extractall(f'{myenv.datadir}2/{symbol}/{interval}/')
				print(f'Extract {file_zip} to {myenv.datadir}2/{symbol}/{interval}/')


		df_result = pd.DataFrame()
		has_csv = False
		file_csv_list = glob.glob(os.path.join(f'{myenv.datadir}2/{symbol}/{interval}/', '*.csv'))
		for file_csv_path in file_csv_list:
			if os.path.isfile(file_csv_path):    		
				print(f'Process {file_csv_path}')
				has_csv = True
				data = pd.read_csv(f'{file_csv_path}', sep=',', names=myenv.all_klines_cols)
				data = utils.parse_type_fields(data, False)
				data = utils.adjust_index(data)
				data['symbol'] = symbol
				data.index = data['open_time']
				data.index.name = 'ix_open_time'
				data.sort_index(inplace=True)
				df_result = pd.concat([df_result, data])
				print(f'File {file_csv_path} concatenated...')

		if has_csv:
			if not os.path.exists(f'{myenv.datadir}/{symbol}/'):
				os.makedirs(f'{myenv.datadir}/{symbol}/')
				print(f'Making directory {myenv.datadir}/{symbol}/')

			print(f'Shape: {df_result.shape} - Min Data: {pd.to_datetime(df_result.index.min(), unit="ms")} - Max Data: {pd.to_datetime(df_result.index.max(), unit="ms")}')
			file_dat_path = f'{myenv.datadir}/{symbol}/{symbol}_{interval}.dat'
			df_result = utils.parse_type_fields(df_result, False)
			df_result = utils.adjust_index(df_result)
			df_result.to_csv(f'{file_dat_path}', sep=';', index=False, compression=dict(method='zip'))
			print(f'File {file_dat_path} saved...')


<built-in method __dir__ of sqlite3.Connection object at 0x7f259e500340>
Process BTCUSDT_1m
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-2020-11.zip to /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-2020-10.zip to /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-2020-08.zip to /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-2023-04.zip to /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-2020-02.zip to /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-2018-02.zip to /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/
Extract /home/marcelo/des/mg_crypto_trader/data2/BTCUSDT/1m/BTCUSDT-1m-202

In [2]:
data_origem = pd.read_csv(f'{myenv.datadir}2/ADAUSDT/1h/ADAUSDT-1h-2018-04.csv', sep=',', names=myenv.all_klines_cols)
data_origem = utils.parse_type_fields(data_origem, False)
data_origem = utils.adjust_index(data_origem)
data_origem['open_time_dt'] = pd.to_datetime(data_origem['open_time'], unit="ms")

data_origem


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,open_time_dt
ix_open_time,,,,,,,,,,,,,
1523937600000,1523937600000,0.25551,0.28800,0.25551,0.26664,8143693.000,1523941199999,2.165078e+06,4421,2.889824e+06,7.671342e+05,0,2018-04-17 04:00:00
1523941200000,1523941200000,0.26660,0.27798,0.26010,0.26200,8317923.500,1523944799999,2.235633e+06,5149,3.819528e+06,1.025247e+06,0,2018-04-17 05:00:00
1523944800000,1523944800000,0.26221,0.26396,0.24800,0.25664,8420095.000,1523948399999,2.153964e+06,4566,3.636154e+06,9.302932e+05,0,2018-04-17 06:00:00
1523948400000,1523948400000,0.25662,0.26300,0.25489,0.25698,4686044.000,1523951999999,1.215621e+06,3283,1.633202e+06,4.234452e+05,0,2018-04-17 07:00:00
1523952000000,1523952000000,0.25636,0.25998,0.25229,0.25631,3510038.250,1523955599999,8.960958e+05,2408,1.349217e+06,3.444861e+05,0,2018-04-17 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525114800000,1525114800000,0.34777,0.34837,0.34435,0.34559,1569923.500,1525118399999,5.433477e+05,1382,5.188862e+05,1.795752e+05,0,2018-04-30 19:00:00
1525118400000,1525118400000,0.34522,0.34744,0.34285,0.34336,1451547.625,1525121999999,5.008790e+05,1248,5.404689e+05,1.864918e+05,0,2018-04-30 20:00:00
1525122000000,1525122000000,0.34336,0.34418,0.33500,0.33531,4260972.000,1525125599999,1.444423e+06,3557,2.086386e+06,7.078376e+05,0,2018-04-30 21:00:00


In [3]:
data_destino = pd.read_csv(f'{myenv.datadir}/ADAUSDT/ADAUSDT_1h.dat', sep=';', compression=dict(method='zip'))
data_destino = utils.parse_type_fields(data_destino, False)
data_destino = utils.adjust_index(data_destino)
data_destino['open_time_dt'] = pd.to_datetime(data_destino['open_time'], unit="ms")
data_destino

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,symbol,open_time_dt
ix_open_time,,,,,,,,,,,,,,
1523937600000,1523937600000,0.25551,0.28800,0.25551,0.26664,8143693.000,1523941199999,2.165078e+06,4421,2.889824e+06,7.671342e+05,0,ADAUSDT,2018-04-17 04:00:00
1523941200000,1523941200000,0.26660,0.27798,0.26010,0.26200,8317923.500,1523944799999,2.235633e+06,5149,3.819528e+06,1.025247e+06,0,ADAUSDT,2018-04-17 05:00:00
1523944800000,1523944800000,0.26221,0.26396,0.24800,0.25664,8420095.000,1523948399999,2.153964e+06,4566,3.636154e+06,9.302932e+05,0,ADAUSDT,2018-04-17 06:00:00
1523948400000,1523948400000,0.25662,0.26300,0.25489,0.25698,4686044.000,1523951999999,1.215621e+06,3283,1.633202e+06,4.234452e+05,0,ADAUSDT,2018-04-17 07:00:00
1523952000000,1523952000000,0.25636,0.25998,0.25229,0.25631,3510038.250,1523955599999,8.960958e+05,2408,1.349217e+06,3.444861e+05,0,ADAUSDT,2018-04-17 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696100400000,1696100400000,0.25430,0.25460,0.25380,0.25390,1265373.375,1696103999999,3.217477e+05,1203,5.567097e+05,1.415727e+05,0,ADAUSDT,2023-09-30 19:00:00
1696104000000,1696104000000,0.25390,0.25420,0.25320,0.25400,1303890.500,1696107599999,3.306864e+05,1052,7.049525e+05,1.788359e+05,0,ADAUSDT,2023-09-30 20:00:00
1696107600000,1696107600000,0.25400,0.25420,0.25290,0.25360,961543.000,1696111199999,2.438040e+05,1223,4.304953e+05,1.091828e+05,0,ADAUSDT,2023-09-30 21:00:00
